In [1]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [2]:
# import libraries
import csv
import pymongo
import pandas as pd
from pathlib import Path

In [3]:
# import csv file
salary_dataset = pd.read_excel("../Datasets/Player_salary_1990-2017.xlsx")
birthplace_dataset = pd.read_csv("../Datasets/final_heatmap.csv")
yearlyBirthplace_dataset = pd.read_csv("../Datasets/final_bar_chart_race_data.csv")

In [4]:
salary_dataset = salary_dataset.rename(columns={'Player Name':'player_name','Salary in $':'salary_$','Season Start':'season_start','Season End':'season_end','Full Team Name':'team_name'})
salary_dataset = salary_dataset.drop(axis=1,columns=['Register Value'])
salary_dataset.head()

,player_name,salary_$,season_start,season_end,Team,team_name
0,A.C. Green,1750000,1990,1991,LAL,Los Angeles Lakers
1,A.C. Green,1750000,1991,1992,LAL,Los Angeles Lakers
2,A.C. Green,1750000,1992,1993,LAL,Los Angeles Lakers
3,A.C. Green,1885000,1993,1994,PHO,Phoenix Suns
4,A.C. Green,6472600,1994,1995,PHO,Phoenix Suns


In [5]:
yearlyBirthplace_dataset

,State,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Alabama,8,9,8,11,8,11,14,12,13,...,4,3,6,8,4,5,5,6,5,5
1,Alaska,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,1,1,0,1
2,Arizona,3,2,1,1,2,1,1,1,0,...,1,1,2,2,1,1,5,4,2,2
3,Arkansas,7,6,5,4,4,6,8,7,8,...,5,5,7,5,6,6,5,7,6,3
4,California,25,29,30,41,30,34,29,34,37,...,44,48,44,53,54,51,66,70,59,49
5,Colorado,1,2,1,1,2,2,2,3,2,...,3,1,1,2,1,1,0,0,0,0
6,Connecticut,6,6,6,8,4,4,3,5,5,...,3,2,2,3,3,3,1,0,2,2
7,Delaware,1,1,1,1,1,1,1,1,0,...,2,1,2,0,0,0,0,0,0,0
8,Florida,11,12,12,13,11,12,14,17,11,...,18,19,16,19,12,11,14,12,18,12
9,Foreign Country,19,19,18,22,17,22,23,27,29,...,60,79,73,77,74,78,107,110,125,99


# Establish connection to MongoDB

In [6]:
# Establish connection to local host 27017
conn = 'mongodb://localhost:27017'

# Connect to MongoClient
client = pymongo.MongoClient(conn)
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [7]:
# Begin by dropping any previous version of the MongoDB instance.
client.drop_database('nba_players_db')

In [8]:
# Create database by naming db here
db = client.nba_players_db

# Create collection by naming collection here

### Salary collection

In [9]:
# Create salary collection
salary_collection = db.player_salary_info

In [10]:
# Use ".to_dict('records')"in combination with .csv variable to create list of dictionaries data set
salary_dataset.reset_index(inplace=True)

salary_dict = salary_dataset.to_dict('records')

# Insert data from the 'data_dict' variable into the collection using 'collection' variable
salary_collection.insert_many(salary_dict)

### Player WS Collection

In [11]:
# Create collection for player WS info 1976-2020
playerWS_collection = db.playerWS


In [12]:
folder = "../Data/Data_Raw"

year = 1976

counter = 0

# i = 0

for file in Path(folder).glob('*.csv'):
    df = pd.read_csv(file)
    df.drop(df[df['Tm'] == 'TOT'].index, inplace = True)
    df = df[['Player','MP','WS', 'WS/48', 'BPM']]
    df = df.groupby(['Player']).sum().reset_index()
    df['Player'] = df['Player'].str.replace('?', '')
    df['Player'] = df['Player'].str.replace('*', '')
    for index, row in df.iterrows():
        obj = {
            'year': year,
            'player': row['Player'],
            'mp': row['MP'],
            'ws': row['WS'],
            'wsper48': row['WS/48'],
            'bpm': row['BPM']
        }
    year += 1   
    counter += 1
    
    
    playerWS_collection.insert_one(obj)

#     i += 1
#     if i == 17:
#        break

In [13]:
# len(obj)
# print(counter)

### Player birthplace collection

In [14]:
player_birthplace_collection = db.playerBirthplace

In [15]:
# Use ".to_dict('records')"in combination with .csv variable to create list of dictionaries data set
birthplace_dataset.reset_index(inplace=True)

birthplace_dict = birthplace_dataset.to_dict('records')

# Insert data from the 'data_dict' variable into the collection using 'collection' variable
player_birthplace_collection.insert_many(birthplace_dict)


### Birthplace with WS and location collection

In [16]:
WS_birthplace_collection = db.WSbirthplace

In [17]:
# Creating an object from the DF
for index, row in birthplace_dataset.iterrows():
    heatmap = {
        'birthplace': row['Birth_Place'],
        'win_shares': row['WS'],
        'long': row['longitude'],
        'lat': row['latitude'], 
    }
    
    WS_birthplace_collection.insert_one(heatmap)

### Birthplace by Year collection

In [18]:
birthplace_by_year_collection = db.yearlyBirthplace

In [19]:
# Use ".to_dict('records')"in combination with .csv variable to create list of dictionaries data set
yearlyBirthplace_dataset.reset_index(inplace=True)

yearlyBirthplace_dict = yearlyBirthplace_dataset.to_dict('records')

# Insert data from the 'data_dict' variable into the collection using 'collection' variable
birthplace_by_year_collection.insert_many(yearlyBirthplace_dict)


In [20]:
# # CODE TO PRINT OUT CONTENTS OF THE DATABASE
# salary_collection = db['player_salary_info']

# result = salary_collection.find( {"Player Name": "LeBron James"} )
# # result

# for doc in result:
#     print (doc)
# ======================================================
# sal = salary_collection.find()
# sal

# for x in sal:
#     print(x)

In [21]:
# After finished with analysis, drop database by uncommenting and running this code
# client.drop_database('final_testing_beds_db')

In [22]:
# <<variable name>> = db.testing_beds.find({"<<field>>":"<<value>>"}).pretty()
